## **Installing the REquired Libraries`**

In [30]:
!pip install opencv-python-headless pytesseract pdf2image pillow

# Install necessary libraries
!pip install opencv-python-headless pytesseract pdf2image pillow
# Install poppler-utils
!apt-get install poppler-utils

# Import necessary modules
from pdf2image import convert_from_path
from google.colab import files

!apt-get install tesseract-ocr
!apt-get install libtesseract-dev

import pytesseract
from PIL import Image

# Use the which command to find the actual path to tesseract
# If you get an empty output, it means tesseract is not in your PATH
!which tesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
/usr/bin/tesseract


## **Load And Prepare the Form**

In [31]:
# Upload the PDF file
uploaded = files.upload()

# Get the filename of the uploaded PDF
pdf_filename = list(uploaded.keys())[0]

# Convert uploaded PDF to images
# The poppler_path argument is added to point to the location of poppler utilities.
# This is usually not needed if poppler is installed and in the system's PATH.
pages = convert_from_path(pdf_filename, dpi=300, poppler_path='/usr/bin') # Pass the filename from uploaded
for i, page in enumerate(pages):
    image_path = f'page_{i + 1}.jpg'
    page.save(image_path, 'JPEG')
    print(f"Saved: {image_path}")

Saving Form 1.pdf to Form 1 (3).pdf
Saved: page_1.jpg
Saved: page_2.jpg
Saved: page_3.jpg


## **Apply OCR to Extract Text**

In [32]:
import pytesseract
from PIL import Image

pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'  # Default path for Colab


In [33]:
extracted_text = ""
for i in range(len(pages)):
    img = Image.open(f'page_{i + 1}.jpg')
    text = pytesseract.image_to_string(img)
    extracted_text += text + "\n"
print(extracted_text)


© >) Enzigma

We start where others quit

Affix Your Photograph Here
CANDIDATE INFORMATION FORM

1. Name (Block Letters- As specified on Passport or Pan Card):
Dr. Girish Anil Amrutkar

 

(First Name) (Middle Name) (Last Name)
2. Permanent Address:

3.1 Street Address: Ganesh Colony.
3.2 City: Jalgaon 3.3 State: Maharashtra

3.4 Zip Code: 425401 3.5 Country: India

3. Current Address:

3.1 Street Address: Dehu Alandi Road

 

3.2 City: Pune 3.3 State: Maharashtra

3.4 Zip Code: 412105 3.5 Country: India

4. Date of Birth: 05 /10 /2003

5. Age: 21 6. Gender: Male

7. Passport: No 8. Mobile: 7785652378

9. PAN No.: D45P189976

10. Visa: No 11. Email ID: girish.amrutkar@mitaoe.ac.in

12. Name of Emergency Contact: Anil Amrutkar

13. Emergency Contact's Number: 9860870788 14. Available for Relocation:

15. EDUCATIONAL QUALIFICATION: (Starting from recent)

 

 

 

 

Sr No. Name of the School/ University Qualification % or Pass out
CGPA Year
1. | MIT Academy Of Engineering BTech 8.16 202

In [34]:
import re
import json

def extract_form_data(text):
    """Extracts data from the given text using regular expressions."""

    name_pattern = r"Name.*?:\s*(.*)"
    email_pattern = r"Email ID.*?:\s*([\w\.-]+@[\w\.-]+)"
    mobile_pattern = r"Mobile.*?:\s*(\d{10})"
    dob_pattern = r"Date of Birth.*?:\s*(\d{2}\s*/\s*\d{2}\s*/\s*\d{4})"


    name = re.search(name_pattern, text)
    email = re.search(email_pattern, text)
    mobile = re.search(mobile_pattern, text)
    dob = re.search(dob_pattern, text)
    # ... (add other extraction logic as needed)

    data = {
        "Name": name.group(1) if name else "Not Found",
        "Email": email.group(1) if email else "Not Found",
        "Mobile": mobile.group(1) if mobile else "Not Found",
        "Date of Birth": dob.group(1) if dob else "Not Found",
        # ... (add other fields as needed)
    }

    return data

# Assuming you have the extracted text in a variable called 'extracted_text'
extracted_data = extract_form_data(extracted_text)

# Convert the extracted data to JSON format
json_data = json.dumps(extracted_data, indent=4)  # indent for better readability

# Print the JSON data or save it to a file
print(json_data)
# or
# with open('extracted_data.json', 'w') as f:
#     f.write(json_data)

{
    "Name": "Dr. Girish Anil Amrutkar",
    "Email": "girish.amrutkar@mitaoe.ac.in",
    "Mobile": "7785652378",
    "Date of Birth": "05 /10 /2003"
}


### **Parse Extracted Text for Relevant Fields**

In [35]:
import re

name_pattern = r"Name.*?:\s*(.*)"
email_pattern = r"Email ID.*?:\s*([\w\.-]+@[\w\.-]+)"
mobile_pattern = r"Mobile.*?:\s*(\d+)"
dob_pattern = r"Date of Birth.*?:\s*(\d{2}\s*/\s*\d{2}\s*/\s*\d{4})"
age_pattern = r"Age:\s*(\d+)"
gender_pattern = r"Gender:\s*(\w+)"
emergency_contact_name_pattern = r"Name of Emergency Contact.*?:\s*(.*)"
emergency_contact_number_pattern = r"Emergency Contact's Number.*?:\s*(\d+)"

# Extract details
name = re.search(name_pattern, extracted_text)
email = re.search(email_pattern, extracted_text)
mobile = re.search(mobile_pattern, extracted_text)
dob = re.search(dob_pattern, extracted_text)
age = re.search(age_pattern, extracted_text)
gender = re.search(gender_pattern, extracted_text)
emergency_contact_name = re.search(emergency_contact_name_pattern, extracted_text)
emergency_contact_number = re.search(emergency_contact_number_pattern, extracted_text)

print("Name:", name.group(1).strip() if name else "Not Found")
print("Email:", email.group(1).strip() if email else "Not Found")
print("Mobile:", mobile.group(1).strip() if mobile else "Not Found")
print("Date of Birth:", dob.group(1).strip() if dob else "Not Found")
print("Age:", age.group(1).strip() if age else "Not Found")
print("Gender:", gender.group(1).strip() if gender else "Not Found")
print("Emergency Contact Name:", emergency_contact_name.group(1).strip() if emergency_contact_name else "Not Found")
print("Emergency Contact Number:", emergency_contact_number.group(1).strip() if emergency_contact_number else "Not Found")

Name: Dr. Girish Anil Amrutkar
Email: girish.amrutkar@mitaoe.ac.in
Mobile: 7785652378
Date of Birth: 05 /10 /2003
Age: 21
Gender: Male
Emergency Contact Name: Anil Amrutkar
Emergency Contact Number: 9860870788


Print the Extracted Data

In [36]:
# Create a dictionary to store extracted data
data = {
    "Name": name.group(1).strip() if name else "Not Found",
    "Email": email.group(1).strip() if email else "Not Found",
    "Mobile": mobile.group(1).strip() if mobile else "Not Found",
    "Date of Birth": dob.group(1).strip() if dob else "Not Found",
    "Age": age.group(1).strip() if age else "Not Found",
    "Gender": gender.group(1).strip() if gender else "Not Found",
    "Emergency Contact Name": emergency_contact_name.group(1).strip() if emergency_contact_name else "Not Found",
    "Emergency Contact Number": emergency_contact_number.group(1).strip() if emergency_contact_number else "Not Found"
}

# Print the extracted data in a structured format
print("Extracted Data:", data)


Extracted Data: {'Name': 'Dr. Girish Anil Amrutkar', 'Email': 'girish.amrutkar@mitaoe.ac.in', 'Mobile': '7785652378', 'Date of Birth': '05 /10 /2003', 'Age': '21', 'Gender': 'Male', 'Emergency Contact Name': 'Anil Amrutkar', 'Emergency Contact Number': '9860870788'}


Save The Extracted Data

In [37]:
import json

# Save as JSON
with open('extracted_data.json', 'w') as f:
    json.dump(data, f)

print("Data saved to extracted_data.json")


Data saved to extracted_data.json
